In [ ]:
pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=efe22d25bcdccf5f3e266fd2ce73076c852e56be98bedaca2394d6d8d4352d31
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales_data')

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT YEAR(date), ROUND(AVG(price),2) FROM home_sales_data WHERE bedrooms = 4 GROUP BY 1").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2019|            300263.7|
|      2020|           298353.78|
|      2021|           301819.44|
+----------+--------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT YEAR(date), ROUND(AVG(price),2) FROM home_sales_data WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY 1").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           292725.69|
|      2019|           287287.82|
|      2020|           294204.16|
|      2021|           294211.46|
+----------+--------------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT YEAR(date), ROUND(AVG(price),2) FROM home_sales_data WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY 1").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           290242.99|
|      2019|           289859.14|
|      2020|           292289.09|
|      2021|           296330.96|
+----------+--------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000?
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_sales_data WHERE price >= 350000 GROUP BY 1").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  16|           399586.53|
+----+--------------------+
only showing top 20 rows

--- 0.8309037685394287 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales_data")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_data')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000.
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_sales_data WHERE price >= 350000 GROUP BY 1").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  16|           399586.53|
+----+--------------------+
only showing top 20 rows

--- 0.646115779876709 seconds ---


The cached runtime of 0.646 seconds is shorter than the uncached runtime of 0.831 seconds.

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_home_sales_p = spark.read.parquet("home_sales_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.
p_home_sales_p.createOrReplaceTempView("p_home_sales_data_p")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM p_home_sales_data_p WHERE price >= 350000 GROUP BY 1").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  73|           752861.18|
|  87|           1072285.2|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  35|           401934.21|
+----+--------------------+
only showing top 20 rows

--- 1.3349974155426025 seconds ---


The runtime on the partitioned formatted parquet data of 1.335 seconds is longer than the cached runtime of 0.646 seconds.  

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales_data")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales_data')

False